<a href="https://colab.research.google.com/github/joyce-leesw/Webscraping/blob/main/top_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import lxml
import re
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive/')

# base_path = "gdrive/MyDrive"
# os.chdir(base_path)

from bs4 import BeautifulSoup
from requests import get

MessageError: ignored

In [ ]:
# MAX_PAGE =51

# imageArr = []

# for i in range(0,MAX_PAGE):
#     totalRecords = 0 if i==0 else (250*i)+1
#     print(totalRecords)
#     url1 = f'https://www.imdb.com/search/title/?title_type=feature,tv_series&count=250&start={totalRecords}&ref_=adv_nxt'
#     imageArr.append(url1)

# imageArr

In [ ]:
os.listdir()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-167-67fdccf289bf>", line 1, in <module>
    os.listdir()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultra

OSError: ignored

In [ ]:
class IMDB(object):
	"""docstring for IMDB"""
	def __init__(self, url):
		super(IMDB, self).__init__()
		page = get(url)

		self.soup = BeautifulSoup(page.content, 'lxml')

	def articleTitle(self):
		return self.soup.find("h1", class_="header").text.replace("\n","")

	def bodyContent(self):
		content = self.soup.find(id="main")
		return content.find_all("div", class_="lister-item mode-advanced")

	def movieData(self):
		movieFrame = self.bodyContent()
		movieTitle = []
		movieDate = []
		movieRunTime = []
		movieGenre = []
		movieRating = []
		movieDirector = []
		movieStars = []
		for movie in movieFrame:
			movieFirstLine = movie.find("h3", class_="lister-item-header")
			movieTitle.append(movieFirstLine.find("a").text)
			movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
			try:
				movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
			except:
				movieRunTime.append(np.nan)
			try:
				movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
			except:
					movieGenre.append(np.nan)
			try:
				movieRating.append(movie.find("strong").text)
			except:
				movieRating.append(np.nan)
			
			shit = movie.find_all("p", class_="")
			try: 
				# shit[1].text.replace("\n","").split('|')[1])
				DirandStar = shit[1].text.replace("\n","").split('|')
				# Stars_string = DirandStar[1].replace("Stars:","")
				Stars_string = DirandStar[1].strip().replace(":",",")
				movieCast_list = Stars_string.split(",")[1:]
				movieStars.append(movieCast_list)
			except:
				movieStars.append(np.nan)

			movieNumbers = movie.find_all("span", attrs={"name": "nv"})
	
		movieData = pd.DataFrame({ "Title": movieTitle, "Release_Year": movieDate, "Genre": movieGenre, "Runtime": movieRunTime, "Ratings": movieRating,"Stars": movieStars,})
		
		# movieData = pd.DataFrame({
    #   "Title": movieTitle,
    #   "Release_Year": movieDate,
    #   "Genre": movieGenre,
    #   "Runtime": movieRunTime,
    #   "Ratings": movieRating, 
		# 	"Stars": movieStars,
		# 	})
		movieData = [movieTitle, movieDate, movieRunTime, movieGenre, movieRating, movieStars]
		return movieData

In [ ]:
# for i in range(len(imageArr)):
#     id1 = IMDB(imageArr[i])
#     print(id1.movieData())

In [ ]:
array_popularity_desc = []
array_popularity_asc = []

array_atoz_desc = []
array_atoz_asc = []

array_userrating_desc = []
array_userrating_asc = []

array_numberofvotes_desc = []
array_numberofvotes_asc = []

array_boxoffice_desc = []
array_boxoffice_asc = []

array_runtime_desc = []
array_runtime_asc = []

# year and release date are similar
array_year_desc = []
array_year_asc = []

array_releasedate_desc = []
array_releasedate_asc = []

array_dateofyourrating_desc = []
array_dateofyourrating_asc = []

array_yourrating_desc = []
array_yourrating_asc = []


In [ ]:
# cant be greater than 40
MAX_PAGE = 40

imageArr = []

year_details = '&year=1931-01-01,2016-12-31'

for i in range(0,MAX_PAGE):
    totalRecords = 0 if i==0 else (250*i)+1
    # print(totalRecords)
    url_pop_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=moviemeter,desc'
    array_popularity_desc.append(url_pop_desc)
    url_pop_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=moviemeter,asc'
    array_popularity_asc.append(url_pop_asc)

    url_atoz_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=alpha,desc'
    array_atoz_desc.append(url_atoz_desc)
    url_atoz_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=alpha,asc'
    array_atoz_asc.append(url_atoz_asc)

    url_userrating_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=user_rating,desc'
    array_userrating_desc.append(url_userrating_desc)
    url_userrating_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=user_rating,asc'
    array_userrating_asc.append(url_userrating_asc)

    url_numvotes_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=num_votes,desc'
    array_numberofvotes_desc.append(url_numvotes_desc)
    url_numvotes_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=num_votes,asc'
    array_numberofvotes_asc.append(url_numvotes_asc)

    url_box_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=boxoffice_gross_us,desc'
    array_boxoffice_desc.append(url_box_desc)
    url_box_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=boxoffice_gross_us,asc'
    array_boxoffice_asc.append(url_box_asc)

    url_runtime_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=runtime,desc'
    array_runtime_desc.append(url_runtime_desc)
    url_runtime_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=runtime,asc'
    array_runtime_asc.append(url_runtime_asc)

    url_year_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=year,desc'
    array_year_desc.append(url_year_desc)
    url_year_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=year,asc'
    array_year_asc.append(url_year_asc)

    url_releasedate_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=release_date,desc'
    array_releasedate_desc.append(url_releasedate_desc)
    url_releasedate_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=release_date,asc'
    array_releasedate_asc.append(url_releasedate_asc)

    url_dateofyourrating_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=your_rating_date,desc'
    array_dateofyourrating_desc.append(url_dateofyourrating_desc)
    url_dateofyourrating_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=your_rating_date,asc'
    array_dateofyourrating_asc.append(url_dateofyourrating_asc)

    url_yourrating_desc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=my_ratings,desc'
    array_yourrating_desc.append(url_yourrating_desc)
    url_yourrating_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=my_ratings,asc'
    array_yourrating_asc.append(url_yourrating_asc)

# print("urls for popularity desc \n")
# print(array_popularity_desc)
# print("urls for popularity asc \n")
# print(array_popularity_asc)

# print("urls for atoz desc \n")
# print(array_atoz_desc)
# print("urls for atoz asc \n")
# print(array_atoz_asc)

# print("urls for userrating desc \n")
# print(array_userrating_desc) 
# print("urls for userrating asc \n")
# print(array_userrating_asc) 

# print("urls for numberofvotes desc \n")
# print(array_numberofvotes_desc)
# print("urls for numberofvotes asc \n")
# print(array_numberofvotes_asc)

# print("urls for boxoffice desc \n")
# print(array_boxoffice_desc) 
# print("urls for boxoffice asc \n")
# print(array_boxoffice_asc) 

# print("urls for runtime desc \n")
# print(array_runtime_desc) 
# print("urls for runtime asc \n")
# print(array_runtime_asc) 

# print("urls for year desc \n")
# print(array_year_desc) 
# print("urls for year asc \n")
# print(array_year_asc) 

# print("urls for releasedate desc \n")
# print(array_releasedate_desc) 
# print("urls for releasedate asc \n")
# print(array_releasedate_asc) 

# print("urls for dateofyourrating desc \n")
# print(array_dateofyourrating_desc)
# print("urls for dateofyourrating asc \n")
# print(array_dateofyourrating_asc)

# print("urls for yourrating desc \n")
# print(array_yourrating_desc)
# print("urls for yourrating asc \n")
# print(array_yourrating_asc)


In [ ]:
all_data_urls = array_popularity_desc + array_popularity_asc + array_atoz_desc + array_atoz_asc + array_userrating_desc + array_userrating_asc + array_numberofvotes_desc + array_numberofvotes_asc + array_boxoffice_desc +array_boxoffice_asc +array_runtime_desc +array_runtime_asc +array_year_desc +array_year_asc +array_releasedate_desc +array_releasedate_asc +array_dateofyourrating_desc +array_dateofyourrating_asc +array_yourrating_desc +array_yourrating_asc 
movieTitle_array, movieDate_array, movieRunTime_array, movieGenre_array, movieRating_array, movieStars_array = [],[],[],[],[],[]

for i in range(len(all_data_urls)):
    id1 = IMDB(all_data_urls[i])
    movieTitle_array = movieTitle_array + id1.movieData()[0]
    movieDate_array = movieDate_array + id1.movieData()[1]
    movieRunTime_array = movieRunTime_array + id1.movieData()[2]
    movieGenre_array = movieGenre_array + id1.movieData()[3]
    movieRating_array = movieRating_array + id1.movieData()[4]
    movieStars_array = movieStars_array + id1.movieData()[5]


messy_movieDf = pd.DataFrame({
    "Title": movieTitle_array,
    "Release_Year": movieDate_array,
    "Run_Time": movieRunTime_array,
    "Genre": movieGenre_array,
    "Rating": movieRating_array,
    "Stars": movieStars_array,
})

In [ ]:
messy_movieDf.shape
# movieDf.drop_duplicates().shape

(199991, 6)

In [ ]:
messy_movieDf

,Title,Release_Year,Run_Time,Genre,Rating,Stars
0,Adi ask bu eziyetin,2010,100,[Drama],6.8,"[Gorgu Altug, Arzu Tan Bayraktutan, Ahmet Be..."
1,Hot Hot Hot,2011,93,"[Comedy, Drama]",4.9,"[Rob Stanley, Joanna Scanlan, Gary Cady, Am..."
2,Eternity and a Day,2016,81,[Drama],NaN,"[Rui Raposo da Silva, Daniela Marques Elias, ..."
3,Lost Pride,2015,62,[Drama],NaN,"[Jide Kene Achufusi, Tana Adelana, Grace Agb..."
4,High School Scandal,1981,90,[Drama],6.9,"[Gina Alajar, Sandy Andolong, Ting Jocson, ..."
...,...,...,...,...,...,...
199986,Gasoline Rainbows,2000,76,[Drama],5.3,"[Jeff Tremper, Kyle Riley, Nora Pierce, Joy..."
199987,Le loup de la côte Ouest,2002,132,"[Crime, Drama, Mystery]",5.9,"[James Faulkner, Anna Mouglalis, Gérard Watk..."
199988,The Nest,2002,107,"[Action, Crime, Thriller]",6.7,"[Samy Naceri, Benoît Magimel, Nadia Farès, ..."
199989,No More Words,1970,78,NaN,NaN,NaN


In [ ]:
movieDf = messy_movieDf.loc[messy_movieDf.astype(str).drop_duplicates().index]
movieDf

NameError: ignored

In [ ]:

# # display(movieDf)
# string = ""
# for i in list(movieDf["Stars"].iloc[int(0.5*len(movieDf)):]):
#   string = string+f"{i}, "
# # type(movieDf)
# print(string)

['Charlie Hewson', ' Eric Edelstein', ' Simone Moore', ' Peter Abbruscato'], ['Dallin Hales', ' James Hales', ' Mallary Madsen', ' Spencer Price'], ['Omkar Das Manikpuri', ' Raghuvir Yadav', ' Shalini Vatsa', ' Farrukh Jaffar'], ['Penelope Bartlau', ' Georgina Capper', ' Ian Dixon', ' Sally McDonald'], ['Adivi Sesh', ' Jade Tailor', ' Sher Ali', ' Veronica Valencia'], ['Lalor Roddy', ' Jonathan Harden', ' Fra Gunn', ' Niall Cusack'], ['Julia Porter Howe', ' Mark Robert Ryan', ' Richard Buonagurio', ' Dee Herlihy'], ['Chris Green', ' Brianne René', ' Scott Bessho', ' Dane Storrusten'], ['Kelly Donohue', ' Andres Londono', ' Liam Kidner', ' Jena Sharma'], ['Veronica Erickson', ' John Hawkes', ' Janet Hurley Kimlicko', ' Zane Rockenbaugh'], ['Alexa Alemanni', ' Christine Avery', ' Russelle Avionn', ' Laura Calhoun'], ['Nao Nagasawa', ' Anthony Laurent', ' Daniel Wilkinson', ' Justin Morck'], ['Ty Barto', ' Jason Benjamin', ' Adam Boyer', ' Destin Bradley'], ['Yun Wei', ' Shaofeng Feng', '